In [829]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from scipy import sparse
from implicit.als import AlternatingLeastSquares
import numpy as np
import statsmodels.api as sm
from sklearn import metrics
from sklearn import svm

In [768]:
fileName = "/Users/rahulkhanna/Downloads/JWPlayer_takehome/jw_player_play_data.csv"

In [769]:
table = pd.read_csv(fileName, header=0)

In [770]:
table.head()
# table.columns

,Unnamed: 0,viewer_id,embed_id,media_id,publisher_category,play_seq,start_time,viewer_tz_offset,watched_duration,watched_pct,device_type,country_code,operating_system,browser,ad_starts,ad_skips,publisher
0,0,87d4ed651d7734062ae1a66774224cb5f4b1b5b5,wxh68oikj3e2,nTrJR9Bp,Arts & Entertainment,1,2017-04-01 16:00:12,-5,50,37.5,Desktop,US,Windows,Chrome,0,0,Publisher 1
1,1,3f32d29cca5be762dfb3c2b0a787a3566efee41b,vwlmthe526hv,MVAUx6Aw,Arts & Entertainment,1,2017-04-01 06:40:46,-4,36,75.0,Desktop,US,Linux,Chrome,0,0,Publisher 1
2,2,3f32d29cca5be762dfb3c2b0a787a3566efee41b,1fui6mc83q3p,MVAUx6Aw,Arts & Entertainment,1,2017-04-01 06:08:52,-4,24,50.0,Desktop,US,Linux,Chrome,0,0,Publisher 1
3,3,1241192f3889451da505044ae9f5ca31330fdeb2,dt64anj5egeh,EwYnAdy0,Arts & Entertainment,1,2017-04-01 07:54:58,-5,11,25.0,Desktop,US,Linux,Chrome,0,0,Publisher 1
4,4,65b973c2c861455cc3115b02d624c93edc0bd9f5,40b1yxz5qrhi,EwYnAdy0,Arts & Entertainment,1,2017-04-01 13:42:12,-4,33,75.0,Desktop,US,Windows,Chrome,0,0,Publisher 1


In [771]:
table['publisher_category'] = table['publisher_category'].astype("category")
table['device_type'] = table['device_type'].astype("category")
table['country_code'] = table['country_code'].astype("category")
table['operating_system'] = table['operating_system'].astype("category")
table['browser'] = table['browser'].astype("category")
table['publisher'] = table['publisher'].astype("category")
table['viewer_id'] = table['viewer_id'].astype("category")
table['media_id'] = table['media_id'].astype("category")

id_pub_cat = dict(enumerate(table['publisher_category'].cat.categories))
pub_cat_ids = {x: i for i, x in id_pub_cat.iteritems()}
print("here are the different publisher_categories")
print(str(pub_cat_ids))

id_device_type = dict(enumerate(table['device_type'].cat.categories))
device_type_ids = {x: i for i, x in id_device_type.iteritems()}
print("here are the different device_types")
print(str(device_type_ids))

id_country_code = dict(enumerate(table['country_code'].cat.categories))
country_code_ids = {x: i for i, x in id_country_code.iteritems()}
print("here are the different country codes")
print(str(country_code_ids))

id_operating_system = dict(enumerate(table['operating_system'].cat.categories))
operating_system_ids = {x: i for i, x in id_operating_system.iteritems()}
print("here are the operating systems")
print(str(operating_system_ids))

id_browser = dict(enumerate(table['browser'].cat.categories))
browser_ids = {x: i for i, x in id_browser.iteritems()}
print("here are the different browsers")
print(str(browser_ids))

id_publisher = dict(enumerate(table['publisher'].cat.categories))
publisher_ids = {x: i for i, x in id_publisher.iteritems()}
print("here are the different publishers")
print(str(publisher_ids))

id_media_id = dict(enumerate(table['media_id'].cat.categories))
media_id_ids = {x: i for i, x in id_media_id.iteritems()}

id_viewer_id = dict(enumerate(table['viewer_id'].cat.categories))
viewer_id_ids = {x: i for i, x in id_viewer_id.iteritems()}

country_code_to_continent = {}
cotinent_mapping = {
    "AF" : 0,
    "AS" : 1,
    "OC" : 2,
    "EU" : 3,
    "NA" : 4,
    "SA" : 5,
    "AN" : 6
}
with open("country_codes_to_continent") as f:
    for line in f:
        split = line.strip().split(",")
        country_code_to_continent[split[0]] = split[1]

here are the different publisher_categories
{'Business & Technology': 1, 'LMS': 3, 'Arts & Entertainment': 0, 'Sports': 9, 'Resellers': 7, 'OVPs': 6, 'OEM': 5, 'News': 4, 'SSPs / SVOD': 8, 'Corporate': 2}
here are the different device_types
{'Phone': 2, 'Other': 1, 'Tablet': 3, 'Desktop': 0}
here are the different country codes
{'BD': 13, 'BE': 14, 'BG': 15, 'BA': 11, 'BB': 12, 'BM': 17, 'BN': 18, 'BO': 19, 'BH': 16, 'BT': 23, 'JM': 79, 'BW': 24, 'WS': 164, 'BQ': 20, 'BR': 21, 'BS': 22, 'JE': 78, 'BY': 25, 'BZ': 26, 'RU': 134, 'RW': 135, 'RS': 133, 'RE': 131, 'RO': 132, 'GU': 63, 'GT': 62, 'GR': 61, 'JP': 81, 'GY': 64, 'GG': 57, 'GF': 56, 'GE': 55, 'GD': 54, 'GB': 53, 'SV': 145, 'GM': 60, 'GI': 59, 'GH': 58, 'OM': 119, 'TN': 150, '--': 0, 'JO': 80, 'HR': 67, 'HU': 68, 'HK': 65, 'HN': 66, 'PR': 126, 'PS': 127, 'PT': 128, 'PY': 129, 'PA': 120, 'PG': 122, 'PE': 121, 'PK': 124, 'PH': 123, 'PL': 125, 'ZM': 167, 'EE': 45, 'EG': 46, 'ZA': 166, 'EC': 44, 'IT': 77, 'ET': 48, 'ZW': 168, 'SA': 13

In [772]:
msk = np.random.rand(len(table)) < 0.8
table_train = table[msk]
table_test = table[~msk]

In [773]:
user_video = table_train[['viewer_id', 'media_id']]

user_video_counts = user_video.groupby(['viewer_id', 'media_id']).size().reset_index()
user_video_counts.columns = ['viewer_id', 'media_id', 'counts']
user_video_counts['viewer_id'] = user_video_counts['viewer_id'].astype("category")
user_video_counts['media_id'] = user_video_counts['media_id'].astype("category")

plays = sparse.csr_matrix((user_video_counts["counts"].astype("float"),
                           (user_video_counts["media_id"].cat.codes.copy(), 
                            user_video_counts["viewer_id"].cat.codes.copy())))

collab_model = AlternatingLeastSquares(factors = 100, iterations = 50)
collab_model.fit(40*plays)

user_video_counts_dic = {}
for row in user_video_counts.to_dict(orient='records'):
    user = row["viewer_id"]
    video = row["media_id"]
    count = row["counts"]
    if user in user_video_counts_dic:
        user_video_counts_dic[user][video] = count
    else:
        user_video_counts_dic[user] = {}
        user_video_counts_dic[user][video] = count


video_counts = user_video.groupby(['media_id']).size().reset_index()
video_counts.columns = ['media_id', 'counts']

video_counts_dic = {}
for row in video_counts.to_dict(orient='records'):
    video_counts_dic[row["media_id"]] = row["counts"]

user_counts = user_video.groupby(['viewer_id']).size().reset_index()
user_counts.columns = ['viewer_id', 'counts']

user_counts_dic = {}
for row in user_counts.to_dict(orient='records'):
    user_counts_dic[row["viewer_id"]] = row["counts"]

In [774]:
len(user_video_counts["media_id"].cat.codes.copy()), len(user_video_counts["viewer_id"].cat.codes.copy())

(227387, 227387)

In [775]:
video_percents = table_train[["media_id", "watched_pct"]]

video_percents_average = video_percents.groupby(["media_id"]).mean().reset_index()
video_percents_average.columns = ['media_id', 'avg']

video_percents_average_dict = {}
for row in video_percents_average.to_dict(orient='records'):
    if row['avg'] and str(row['avg']) != 'nan':
        video_percents_average_dict[row['media_id']] = row['avg']
    else:
        video_percents_average_dict[row['media_id']] = 0

video_percents_std = video_percents.groupby(["media_id"]).std().reset_index()
video_percents_std.columns = ['media_id', 'std']

video_percents_std_dict = {}
for row in video_percents_std.to_dict(orient='records'):
    if row['std'] and str(row['std']) != 'nan':
        video_percents_std_dict[row['media_id']] = row['std']
    else:
        video_percents_std_dict[row['media_id']] = 0

video_percents_median = video_percents.groupby(["media_id"]).median().reset_index()
video_percents_median.columns = ['media_id', 'median']

video_percents_median_dict = {}
for row in video_percents_median.to_dict(orient='records'):
    if row['median'] and str(row['median']) != 'nan':
        video_percents_median_dict[row['media_id']] = row['median']
    else:
        video_percents_median_dict[row['media_id']] = 0

In [776]:
user_percents = table_train[["viewer_id", "watched_pct"]]

user_percents_average = user_percents.groupby(["viewer_id"]).mean().reset_index()
user_percents_average.columns = ['viewer_id', 'avg']

user_percents_average_dict = {}
for row in user_percents_average.to_dict(orient='records'):
    if row['avg'] and str(row['avg']) != 'nan':
        user_percents_average_dict[row['viewer_id']] = row['avg']
    else:
        user_percents_average_dict[row['viewer_id']] = 0

user_percents_std = user_percents.groupby(["viewer_id"]).std().reset_index()
user_percents_std.columns = ['viewer_id', 'std']

user_percents_std_dict = {}
for row in user_percents_std.to_dict(orient='records'):
    if row['std'] and str(row['std']) != 'nan':
        user_percents_std_dict[row['viewer_id']] = row['std']
    else:
        user_percents_std_dict[row['viewer_id']] = 0

user_percents_median = user_percents.groupby(["viewer_id"]).median().reset_index()
user_percents_median.columns = ['viewer_id', 'median']

user_percents_median_dict = {}
for row in user_percents_median.to_dict(orient='records'):
    if row['median'] and str(row['median']) != 'nan':
        user_percents_median_dict[row['viewer_id']] = row['median']
    else:
        user_percents_median_dict[row['viewer_id']] = 0

In [777]:
with open("temp", "w") as f:
    for row in user_percents.to_dict(orient='records'):
        tup = (row["viewer_id"], row["watched_pct"], user_percents_average_dict[row["viewer_id"]])
        f.write(str(tup))
        f.write("\n")

In [778]:
similar_items = {}
def fillSimilarItems():
    for media_id in media_id_ids:
        similar_items[media_id] = set([id_media_id[v] for v,s in collab_model.similar_items(media_id_ids[media_id], N=100)])

fillSimilarItems()

In [779]:
# using global variables
def intersectingWatchHistoryWithSimilarItems(user_id, media_id):
    if user_id in user_video_counts_dic and media_id in similar_items:
        user_history = set(user_video_counts_dic[user_id].keys())
        if len(user_history):
            return len(similar_items[media_id].intersection(user_history)) / float(len(similar_items[media_id]))
    else:
        return 0
    
    
intersectingWatchHistoryWithSimilarItems("d4614f09cddefb1ff996543c1b82fcbd65fdfdc9","2M8ZBJet")   

0.01

In [917]:
def createFeature(row):
    feature = []
    # 1
    feature.append(row["ad_skips"])
    # 2
    if row["browser"] in browser_ids:
        feature.append(browser_ids[row["browser"]])
    else:
        feature.append(-1)
    # 3
    if row["country_code"] in country_code_ids:
        feature.append(country_code_ids[row["country_code"]])
    else:
        feature.append(-1)
    # 4
    if row["device_type"] in device_type_ids:
        feature.append(device_type_ids[row["device_type"]])
    else:
        feature.append(-1)
        
    # 5
    feature.append(media_id_ids[row["media_id"]])
    
    # 6
    feature.append(viewer_id_ids[row["viewer_id"]])
    
    # 7
    if row["operating_system"] in operating_system_ids:
        feature.append(operating_system_ids[row["operating_system"]])
    else:
        feature.append(-1)

    # 8
    feature.append(row["play_seq"])
    
    # 9
    if row["publisher"] in publisher_ids:
        feature.append(publisher_ids[row["publisher"]])
    else:
        feature.append(-1)

    # 10
    if row["publisher_category"] in pub_cat_ids:
        feature.append(pub_cat_ids[row["publisher_category"]])
    else:
        feature.append(-1)
        
    # 11
    time = datetime.strptime(row["start_time"], '%Y-%m-%d %H:%M:%S')
    feature.append((time-datetime(1970,1,1)).total_seconds())
    
    # 12
    if row["viewer_id"] in user_video_counts_dic and row["media_id"] in user_video_counts_dic[row["viewer_id"]]:
        feature.append(user_video_counts_dic[row["viewer_id"]][row["media_id"]])
    else:
        feature.append(0)

    # 13
    if row["media_id"] in video_counts_dic:
        feature.append(video_counts_dic[row["media_id"]])
    else:
        feature.append(0)
    
    # 14
    if row["country_code"] in country_code_to_continent:
        feature.append(cotinent_mapping[country_code_to_continent[row["country_code"]]])
    else:
        feature.append(-1)
    
    # 15
    feature.append(intersectingWatchHistoryWithSimilarItems(row["viewer_id"], row["media_id"]))
    
#     # 16
#     if user_percents_average_dict[row["viewer_id"]] > 0 :
#         feature.append(user_percents_std_dict[row["viewer_id"]] / user_percents_average_dict[row["viewer_id"]])
#     else:
#         feature.append(0)
    
     # 17
    feature.append(user_percents_std_dict[row["viewer_id"]])
    
     # 18
#     feature.append(user_percents_median_dict[row["viewer_id"]])

    # 19
#     feature.append(user_percents_average_dict[row["viewer_id"]])

    # 20
#     if video_percents_average_dict[row["media_id"]] > 0 :
#         feature.append(video_percents_std_dict[row["media_id"]] / video_percents_average_dict[row["media_id"]])
#     else:
#         feature.append(0)

    # 21
    feature.append(video_percents_std_dict[row["media_id"]])

    # 22
#     feature.append(video_percents_median_dict[row["media_id"]])

    # 23
#     feature.append(video_percents_average_dict[row["media_id"]])

    # 24
    feature.append(user_counts_dic[row["viewer_id"]])


    
    return feature
                   
X_train = []
y_train = []
for row in table_train.to_dict(orient='records'):
    X_train.append(createFeature(row))
    y_train.append(row["watched_pct"])

X_test = []
y_test = []
for row in table_test.to_dict(orient='records'):
    X_test.append(createFeature(row))
    y_test.append(row["watched_pct"])

In [918]:
len(X_train), len(y_train), len(X_test), len(y_test)

(302863, 302863, 75639, 75639)

In [919]:
# X_train, X_test, y_train, y_test = train_test_split(features, percentages, test_size=0.33, random_state=1)

In [920]:
regressor = RandomForestRegressor(n_estimators = 50)

In [921]:
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [922]:
regressor.score(X_test, y_test)

0.31828676132154665

In [925]:
# https://stackoverflow.com/questions/42033720/python-sklearn-multiple-linear-regression-display-r-squared
yhat = regressor.predict(X_test)
SS_Residual = sum((y_test-yhat)**2)
SS_Total = sum((y_test-np.mean(y_test))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y_test)-1)/float((len(y_test)-len(X_test[0])-1))
print r_squared, adjusted_r_squared

0.318286761322 0.318124491575


In [ ]:
regressor.feature_importances_

In [814]:
regressor1 = sm.OLS(y_train, X_train)

In [825]:
results = regressor1.fit_regularized()

In [828]:
print results.summary()

None


In [826]:
results2 = regressor1.predict(results.params, X_test)

In [827]:
metrics.r2_score(y_test, results2)

0.18396935926088043

In [839]:
regressor2 = svm.LinearSVR()

In [841]:
len(X_train[0]), len(X_test[0])

(11, 11)

In [846]:
regressor2.fit(X_train, y_train)

LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [847]:
regressor2.score(X_test, y_test)

0.025138075003707017